In [2]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os 
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import itertools

from multiprocessing import Pool


C:\Users\jhroe\AppData\Local\Temp\ipykernel_19496\1407334459.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:


# date and cell cutoff
cutoff = 25

# specify paths

data_path = f'{os.getcwd()}/Data/'
file_path = f"{data_path}/distances/cell_distances_25px_20240506_allpatients.csv"
file_path2 = f"{data_path}/clustering/full_cd_clus_19_39_48.csv"

# output_path = f"{data_path}/frequencies/"
comms = pd.read_csv(file_path2)[['agglomerateto_39', 'source_ID']]

temp = pd.read_csv(file_path)
# temp
dists = temp[['Patient_ID','source_cluster', 'target_cluster', 'source_ID']]
dists_merged = pd.merge(dists, comms, on='source_ID', how='inner').rename(columns={'agglomerateto_39':'cluster'})

dists_merged['Patient_ID' , 'source_ID', 'source_cluster', 'target_cluster']
# comms

KeyError: ('Patient_ID', 'source_ID', 'source_cluster', 'target_cluster')

In [17]:
len(dists_merged[dists_merged['source_ID'] == 'D223_1836'])

13

In [128]:
# set number of permutations
num_perms = 5
# Significance threshold
p_threshold = 0.01
np.random.seed(42)

# calculate mean number of celltype B in surrounding of celltype A's per treatment, ROI and community
def aggregate_histo(data):
    counts = data.groupby(by=['Patient_ID' , 'source_ID', 'source_cluster', 'target_cluster']).size().reset_index(name='mean')
    means = counts.groupby(by=['Patient_ID' , 'source_cluster', 'target_cluster'])['mean'].mean().reset_index()
    return means

# Shuffle the cell types B in a ROI to randomize the neighbours of cell type A
def shuffle_labels(data):
    shuffled_data = data.copy()
    new_dfs = []
    for roi in shuffled_data['ROI_ID'].unique():
        df = shuffled_data[(shuffled_data['treatment'] == treatment) & (shuffled_data['ROI_ID'] == roi)].copy()
        df['target_cluster'] = np.random.permutation(df['target_cluster'])
        new_dfs.append(df)
    new_df = pd.concat(new_dfs)
    return new_df


# Shuffle the data then obtain mean of number of neighbours per celltype
# Only use for permutation
def shuffle_and_aggregate(data):
    data_shuffled = shuffle_labels(data)
    return aggregate_histo(data_shuffled)


def calc_p_vals(dat_baseline, dat_perm, n_perm, p_thresh=0.01):
    # Merge data
    dat_perm = pd.merge(dat_perm, dat_baseline[['treatment','ROI_ID', 'community','source_cluster', 'target_cluster', 'mean']],
                        on=['treatment','ROI_ID', 'community','source_cluster', 'target_cluster'], suffixes=('_perm', '_obs'), how='left')

    # Replace NA values with 0
    dat_perm['mean_obs'].fillna(0, inplace=True)

    dat_perm_res = dat_perm.groupby(by=['treatment','ROI_ID', 'community','source_cluster', 'target_cluster', 'mean_obs'])['mean_perm'].mean().reset_index()

    # Initialize p-values list
    p_values = []

    # Iterate over each row of dat_perm
    for index, row in dat_perm.iterrows():
        count_obs = row['mean_obs']
        count_perm = row['mean_perm']

        # Statistical calculation
        if count_obs == 0:
            p_gt = 1
        else:
            p_gt = (np.sum(count_perm >= count_obs) + 1) / (n_perm + 1)
        p_lt = (n_perm - np.sum(count_perm > count_obs) + 1) / (n_perm + 1)

        # Direction calculation
        direction = p_gt < p_lt

        # Final p-value calculation
        p = p_gt if direction else p_lt

        # Significance testing
        sig = p < p_thresh

        # Significance values
        sigval = int(sig) * np.sign(int(direction) - 0.5)

        # Append results to p_values list
        # ['treatment','ROI_ID', 'community','source_cluster', 'target_cluster']
        p_values.append([row['treatment'], row['ROI_ID'], row['community'],row['source_cluster'],row['target_cluster'], p_gt, p_lt, direction, p, sig, sigval])

    # Create DataFrame from p_values list
    columns = ['treatment','ROI_ID', 'community','source_cluster', 'target_cluster', 'p_gt', 'p_lt', 'direction', 'p', 'sig', 'sigval']
    dat_stat = pd.DataFrame(p_values, columns=columns)

    return dat_stat, dat_perm_res


perms = []
for i in range(num_perms):
    perms.append(shuffle_and_aggregate(dists_merged))

perm = pd.concat(perms, ignore_index=True)
bas = aggregate_histo(dists_merged)
# shuf = shuffle_and_aggregate(dists_merged)
result, dat_perm_res = calc_p_vals(bas, perm, num_perms, p_thresh=p_threshold)



In [ ]:
result.to_csv("permutation_results_500perm.csv")

In [127]:
result['sigval'].unique()

array([-0.])

In [124]:
dat_perm_res

,treatment,ROI_ID,community,source_cluster,target_cluster,mean_obs,mean_perm
0,ETP,06_ETP,1,B cells,B cells,0.000000,2.000000
1,ETP,06_ETP,1,B cells,Dendritic cells,3.000000,1.000000
2,ETP,06_ETP,1,B cells,Endothelium,0.000000,1.500000
3,ETP,06_ETP,1,B cells,Epithelium,0.000000,2.000000
4,ETP,06_ETP,1,B cells,Fibroblasts,0.000000,2.000000
...,...,...,...,...,...,...,...
16295,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Macrophages type2,1.200000,1.666667
16296,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Neutrophils,1.000000,1.125000
16297,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,T cells CD4,1.000000,2.125000
16298,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,T cells CD8,2.000000,1.000000


In [108]:
perms[0]

,treatment,ROI_ID,community,source_cluster,target_cluster,mean
0,ETP,06_ETP,1,B cells,B cells,2.000000
1,ETP,06_ETP,1,B cells,Endothelium,3.000000
2,ETP,06_ETP,1,B cells,Fibroblasts,2.000000
3,ETP,06_ETP,1,B cells,Macrophages type1,2.000000
4,ETP,06_ETP,1,B cells,Macrophages type2,1.000000
...,...,...,...,...,...,...
15258,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Macrophages type2,2.000000
15259,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Neutrophils,1.333333
15260,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,T cells CD4,2.666667
15261,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,T cells CD8,1.000000


In [97]:
result[(result['direction'] == False)]

,treatment,ROI_ID,community,source_cluster,target_cluster,p_gt,p_lt,direction,p,sig,sigval
0,ETP,06_ETP,1,B cells,B cells,1.0,0.998004,False,0.998004,False,-0.0
1,ETP,06_ETP,1,B cells,Dendritic cells CD103,1.0,0.998004,False,0.998004,False,-0.0
2,ETP,06_ETP,1,B cells,Endothelium,1.0,0.998004,False,0.998004,False,-0.0
3,ETP,06_ETP,1,B cells,Fibroblasts,1.0,0.998004,False,0.998004,False,-0.0
4,ETP,06_ETP,1,B cells,Macrophages type1,1.0,0.998004,False,0.998004,False,-0.0
...,...,...,...,...,...,...,...,...,...,...,...
7642554,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,T cells CD8,Macrophages type2,1.0,0.998004,False,0.998004,False,-0.0
7642556,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,T cells CD8,T cells CD4,1.0,0.998004,False,0.998004,False,-0.0
7642560,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Dendritic cells,1.0,0.998004,False,0.998004,False,-0.0
7642561,MRTX+PD1+CTLA-4,07_MRTX+PD1+CTLA-4,20,Tumour,Dendritic cells CD103,1.0,0.998004,False,0.998004,False,-0.0


In [9]:
# # for celltype in dists['source_cluster'].unique()
# celltype = 'T cells CD8'
# num_permutations = 10
# ct_df = dists_merged[dists_merged['source_cluster'] == celltype]
# for community in ct_df['community'].unique():
#     for ROI in ct_df['ROI_ID'].unique():
#         # for run in num_permutations:
            
#         roi_df = ct_df[ct_df['ROI_ID'] == ROI]
#         counts_roi_df = roi_df.groupby(by=['source_ID','ROI_ID', 'source_cluster', 'target_cluster', 'community']).size().reset_index(name='count')
#         counts_roi_df = counts_roi_df.groupby(by=['ROI_ID', 'source_cluster', 'target_cluster', 'community'])['count'].mean().reset_index()

# counts_roi_df

,ROI_ID,source_cluster,target_cluster,community,count
0,08_MRTX+PD1,T cells CD8,B cells,1,1.000000
1,08_MRTX+PD1,T cells CD8,B cells,2,1.000000
2,08_MRTX+PD1,T cells CD8,B cells,3,2.342857
3,08_MRTX+PD1,T cells CD8,B cells,5,2.000000
4,08_MRTX+PD1,T cells CD8,B cells,6,1.500000
...,...,...,...,...,...
145,08_MRTX+PD1,T cells CD8,Tumour,10,4.000000
146,08_MRTX+PD1,T cells CD8,Tumour,11,8.352273
147,08_MRTX+PD1,T cells CD8,Tumour,14,2.000000
148,08_MRTX+PD1,T cells CD8,Tumour,15,3.000000


In [10]:
# df = counts_roi_df.copy()


# # Get unique values for each category
# cat1_values = dists_merged['ROI_ID'].unique()
# cat2_values = dists_merged['source_cluster'].unique()
# cat3_values = dists_merged['target_cluster'].unique()
# cat4_values = dists_merged['community'].unique()

# # Generate all combinations of categorical values
# all_combinations = list(itertools.product(cat1_values, cat2_values, cat3_values, cat4_values))

# # Create a new dataframe with all combinations
# df_all_combinations = pd.DataFrame(all_combinations, columns=['ROI_ID', 'source_cluster', 'target_cluster', 'community'])

# # Merge the original dataframe with the new dataframe
# perm_test_df = pd.merge(df_all_combinations, df, on=['ROI_ID', 'source_cluster', 'target_cluster', 'community'], how='left')

# # Fill NaN values in 'Count' column with 0
# perm_test_df['count'].fillna(0, inplace=True)

# perm_test_df['total'] = 0
# perm_test_df['higher'] = 0
# perm_test_df['tie'] = 0

# perm_test_df['lower'] = 0
# perm_test_df


C:\Users\jhroe\AppData\Local\Temp\ipykernel_20552\933639490.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  perm_test_df['count'].fillna(0, inplace=True)


,ROI_ID,source_cluster,target_cluster,community,count,total,higher,tie,lower
0,04_MRTX+PD1+CTLA-4,Endothelium,B cells,3,0.0,0,0,0,0
1,04_MRTX+PD1+CTLA-4,Endothelium,B cells,10,0.0,0,0,0,0
2,04_MRTX+PD1+CTLA-4,Endothelium,B cells,20,0.0,0,0,0,0
3,04_MRTX+PD1+CTLA-4,Endothelium,B cells,16,0.0,0,0,0,0
4,04_MRTX+PD1+CTLA-4,Endothelium,B cells,8,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
28387,08_MRTX+PD1,T reg cells,T reg cells,21,0.0,0,0,0,0
28388,08_MRTX+PD1,T reg cells,T reg cells,17,0.0,0,0,0,0
28389,08_MRTX+PD1,T reg cells,T reg cells,18,0.0,0,0,0,0
28390,08_MRTX+PD1,T reg cells,T reg cells,2,0.0,0,0,0,0


In [85]:
# perm_test_df = pd.merge(perm_test_df.iloc[:,:-1], counts_roi_df_rand[['ROI_ID', 'source_cluster', 'target_cluster', 'community', 'count_random']], on=['ROI_ID', 'source_cluster', 'target_cluster', 'community'], how='inner')
# perm_test_df

In [138]:
# def increment_columns(row):
#     if row['count'] > row['count_random']:
#         row['higher'] += 1  # Increment 'z' column
#     elif row['count'] < row['count_random']:
#         row['lower'] += 1  # Increment another column
#     else:
#         row['tie'] += 1  # Increment another column
#     row['total'] += 1
#     return row

In [ ]:
def single_perm(args):
    

In [175]:
# num_permutations = 2
# celltype = 'T cells CD8'

# dists_randomized
# for run in range(num_permutations):
#     dists_randomized = dists_merged.copy()
#     dists_randomized['target_cluster'] = dists_randomized['target_cluster'].sample(frac=1).reset_index(drop=True)

#     # for celltype in dists['source_cluster'].unique()
#     ct_df = dists_randomized[dists_randomized['source_cluster'] == celltype]
#     for community in ct_df['community'].unique():
#         for ROI in ct_df['ROI_ID'].unique():
#             # for run in num_permutations:
#             roi_df_rand = ct_df[ct_df['ROI_ID'] == ROI]
#             counts_roi_df_rand = roi_df_rand.groupby(by=['source_ID','ROI_ID', 'source_cluster', 'target_cluster', 'community']).size().reset_index(name='count_random')
#             counts_roi_df_rand = counts_roi_df_rand.groupby(by=['ROI_ID', 'source_cluster', 'target_cluster', 'community'])['count_random'].mean().reset_index()
#             print(perm_test_df.head(), 'before_norm')
#             print(counts_roi_df_rand[['ROI_ID', 'source_cluster', 'target_cluster', 'community', 'count_random']].head(), 'before_rand')

#             perm_test_df = pd.merge(perm_test_df[['ROI_ID', 'source_cluster', 'target_cluster', 'community', 'count', 'total', 'higher', 'tie', 'lower']], counts_roi_df_rand[['ROI_ID', 'source_cluster', 'target_cluster', 'community', 'count_random']], on=['ROI_ID', 'source_cluster', 'target_cluster', 'community'], how='inner')
#             # if perm_test_df.empty:
#             #     print('ua')
#             #     break
#             print(perm_test_df.head(), 'after')
#             perm_test_df = perm_test_df.apply(increment_columns, axis=1)

#             perm_test_df = perm_test_df.drop(columns=['count_random'])
#     # check if counts_roi_df_rand['count_random'] is high/low than perm_test_df['count]

            
# perm_test_df

               ROI_ID source_cluster target_cluster  community  count  total  \
0  04_MRTX+PD1+CTLA-4    Endothelium        B cells          3    0.0      0   
1  04_MRTX+PD1+CTLA-4    Endothelium        B cells         10    0.0      0   
2  04_MRTX+PD1+CTLA-4    Endothelium        B cells         20    0.0      0   
3  04_MRTX+PD1+CTLA-4    Endothelium        B cells         16    0.0      0   
4  04_MRTX+PD1+CTLA-4    Endothelium        B cells          8    0.0      0   

   higher  tie  lower  
0       0    0      0  
1       0    0      0  
2       0    0      0  
3       0    0      0  
4       0    0      0   before_norm
               ROI_ID source_cluster target_cluster  community  count_random
0  04_MRTX+PD1+CTLA-4    T cells CD8        B cells          1      1.320000
1  04_MRTX+PD1+CTLA-4    T cells CD8        B cells          3      1.166667
2  04_MRTX+PD1+CTLA-4    T cells CD8        B cells          6      1.000000
3  04_MRTX+PD1+CTLA-4    T cells CD8        B cells    

,ROI_ID,source_cluster,target_cluster,community,count,total,higher,tie,lower
